In [5]:
import csv
import datetime
import time
import os

import matplotlib.pyplot as plt
import collections
from openpyxl import load_workbook

import math
import statistics
import numpy as np
import pandas as pd

In [4]:
def get_filename_from_folder(folder_path, ext=""):
    file_name_list = []
    for file_name in os.listdir(folder_path):
        name = os.path.join(folder_path + "/"+ file_name)
        
        # get file extension
        file_ext = os.path.splitext(file_name)[1]
        # if user give ext and the file extension is not the same, pass
        if ext and file_ext != ext:
            continue
        
        # os.rename(name, os.path.join(folder_path + "/"+ file_name[1:]))
    
        file_name_list.append(name)
        
    return sorted(file_name_list)


def filename_process(target, filename_list):
    
    split_file_name = [ x.split("_") for x in filename_list]
    
    for i in split_file_name:
        try:
            ytr = i[1][1:]
            types = i[3]
            start = i[4]
            end = i[5].split(".")[0]
        
            if target == ytr:
                yield ytr, types, start, end
        except:
            pass


def get_file_by_ytr_and_date(ytr, types, start, filename_list):
    file_list = []
    for fytr, ftypes, fstart, fend in filename_process(ytr, filename_list):
        if fytr == ytr and ftypes == types and fstart == start:
            name = f"./yt_/{fytr}_單主題_{ftypes}_{fstart}_{fend}.xlsx"

            file_list.append(name)
    return file_list


def string_to_datetime(s):
    return datetime.datetime.strptime(s, '%Y/%m/%d')


# opview 資料（趨勢 和 好感度）
def get_ytrs_view_and_emotion_by_startdate(ytrs: list, startdate: str, enddate: str, filename_list_op: list):
    
    ytrs_view = []
    ytrs_emotion = []
    
    for ytr in ytrs: # iter each youtuber
        # read all views data
        op_view_data = collections.OrderedDict()
        for date in DATE:
            for filename in get_file_by_ytr_and_date(ytr, '趨勢', date, filename_list_op):
                wb = load_workbook(filename=filename)
                # op_view_data['name'] = ytr
                opview_wb = wb['日趨勢']
                for i in range(2, opview_wb.max_row+1):
                    day = opview_wb.cell(column=1, row=i).value
                    qty = opview_wb.cell(column=2, row=i).value
                    
                    if day >= startdate and day <= enddate:
                        op_view_data[day] = qty
        ytrs_view.append(op_view_data)

        # read all good bad data
        op_view_emotion = collections.OrderedDict(
            {'good': collections.OrderedDict(), 'bad': collections.OrderedDict()})
        for date in DATE:
            for filename in get_file_by_ytr_and_date(ytr, '情緒', date, filename_list_op):
                # opview 資料（情緒）
                wb = load_workbook(filename=filename)
                opview_wb = wb['日情緒趨勢']
                for i in range(2, opview_wb.max_row+1):

                    day = opview_wb.cell(column=1, row=i).value
                    good = opview_wb.cell(column=2, row=i).value
                    bad = opview_wb.cell(column=3, row=i).value
                    total = opview_wb.cell(column=4, row=i).value
                    if day >= startdate and day <= enddate:
                        op_view_emotion["good"][day] = good
                        op_view_emotion["bad"][day] = bad
        ytrs_emotion.append(op_view_emotion)

    return ytrs_view ,ytrs_emotion


# plot views and good/bad emotions
def plot_view_and_emotion(ytrs_view, ytrs_emotion, ytrs, stepsize=1, figsize=(19,4)):
    for view, emotion, name in zip(ytrs_view, ytrs_emotion, ytrs):
        print(name)
        fig, ax1 = plt.subplots(figsize=figsize)
        ax1.plot(view.values(), figure=fig,
                 linewidth=1, c='black', label="opview")
        ax1.plot(emotion["good"].values(), figure=fig,
                 linewidth=1, c='g', label="good")
        ax1.plot(emotion["bad"].values(), figure=fig,
                 linewidth=1, c='r', label="bad")
        plt.xticks(range(0, len(list(view.keys())), stepsize),
                       list(view.keys())[::stepsize], rotation=20)
        ax1.legend()


def get_news_filename_list_by_ytrs(ytrs, data_folder_path, include_news=["ettoday_keywords.csv", "apple_news.csv", "dcard.csv" "ettoday_tag.csv", "ptt.csv"]):
    filename_list_news_all = []

    for ytr in ytrs:
        print(ytr)
        ytr_news = []  # news list for each ytr
        filename_list_news = get_filename_from_folder(
            data_folder_path + ytr)
        for x in filename_list_news:
            if x.split('/')[-1] in include_news:
                ytr_news.append(x)
        filename_list_news_all.append(ytr_news)

    return filename_list_news_all


def get_news_list_by_filename_list(filename_list_news_et):
    news_list = []
    for filename_list in filename_list_news_et:  # iterate ytr
        news = []
        for filename in filename_list:
            with open(filename, encoding="utf-8") as f:
                reader = csv.reader(f)
                for idx, row in enumerate(reader):
                    # Source,Title,Link,Date,Summary,Text,Like

                    if idx == 0 or row == ['Source', 'Title', 'Link', 'Date', 'Summary', 'Text', 'Like']:
                        continue
                    Source = row[0]
                    Title = "".join(row[1].split())
                    Link = row[2]
                    Date = row[3].split(" ")[0].replace('-', '/')
                    Summary = "".join(row[4].split())
                    Text = row[5]

                    news.append({"source": Source, "title": Title, "link": Link,
                                 "date": Date, "summary": Summary, "text": Text})
        news_list.append(news)
    return news_list


def compare_news_list_and_event_list(multiply=3):
    print('單日大於', multiply, '倍過去七天的平均：')

    # event definition
    for view, emotion, name, news in zip(ytrs_view, ytrs_emotion, ytrs, news_list):
        
        print(name)
        # event definition
        data = [list(view.values())[0]] * 7
        data.extend(list(view.values()))

        count = 0
        event_list = []

        for i, d in enumerate(data[7:], start=7):
            seven_day = data[i-7:i]
            seven_day_avg = sum(seven_day)/7
            if d > multiply * seven_day_avg:
                day = list(view.keys())[i-7]
                event_list.append(day)
        print(event_list)

        # find news
        print('來源     事件日期      日期好壞比  新聞日期     新聞日期好壞比    新聞標題')
        for i in news:
            day_add = datetime.timedelta(days=2)
            try:
                news_date = string_to_datetime(i['date'])
                
            except:
                news_date = i['date'].split(" ")[0].replace("-", "/")
                news_date = string_to_datetime(news_date)

            for event in event_list:
                temp_event = event
                temp_view = view
                temp_emotion = emotion
                event_day = string_to_datetime(event)
                if news_date >= event_day and news_date <= event_day + day_add:
                    try:
                        print(i['source'], event_day.strftime('%Y-%m-%d'), ' ',
                            '%.2f' % (round(
                                emotion['good'][event] / emotion['bad'][event], 2)), '    ',
                            news_date.strftime('%Y-%m-%d'), '  '
                            '%.2f' % (round(
                                emotion['good'][i['date']] / emotion['bad'][i['date']], 2)), '       ',
                            i['title'])
                    except:
                        pass


def generate_ytr_name_list(filename="Youtuber名單 - 道歉.csv"):
    ytr_list = []
    with open(filename, 'r') as f:
        rows = csv.reader(f)
        for row in rows:
            if row[1] == "":
                pass
            if row[1] == "頻道(官方帳號)":
                continue
            ytr_name = []
            ytr_name.append(row[1].replace(
                "/", " ").replace("   ", " ").split("\n")[0])

            ytr_name.extend(row[3].replace("、", "\n").split("\n"))
            ytr_list.append(list(dict.fromkeys(ytr_name)))
    return ytr_list
                    


In [313]:
merged_with_bobo = pd.read_csv("../final_with_dropv3.csv")
sorry_event = pd.read_excel("../Youtuber道歉事件.xlsx",
                   sheet_name="bobo new strategy", header=1).iloc[:70]
ytr_type = pd.read_excel("../Youtuber名單_ytr_final_list.xlsx")
merged_with_bobo.columns, sorry_event.columns, ytr_type.columns


(Index(['Youtuber', '期間', '正面情緒', '負面情緒', '正面情緒(前一天)', '正面情緒(前二天)', '正面情緒(後一天)',
        '正面情緒(後二天)', '負面情緒(前一天)', '負面情緒(前二天)', '負面情緒(後一天)', '負面情緒(後二天)',
        '當天正負情緒比', '前一天正負情緒比', '前二天正負情緒比', '後一天正負情緒比', '後二天正負情緒比', '當天情緒比變化量',
        '隔天情緒比變化量', '前一後一情緒變化量', '前二後二情緒變化量', '聲量', '前一天聲量', '前二天聲量', '後一天聲量',
        '後二天聲量', '當天聲量變化量', '隔天聲量變化量', '前一後一聲量變化量', '前二後二聲量變化量', 'Dcard',
        'Dcard內文', 'PTT', 'PTT內文', 'apple_news', 'apple_news內文', 'ettoday',
        'ettoday內文', 'Mobile01', 'Mobile01內文', 'opview', 'g/b', 'sub',
        'category', 'sorry'],
       dtype='object'),
 Index(['youtuber', 'date', '犯錯的類別', '表達目標人物/觀眾的感謝或諒解', '補償', '有沒有道歉', '解釋',
        '引發同情(說自己有病)', '藉口', '犯錯的類別.1', '表達目標人物/觀眾的感謝或諒解.1', '補償.1', '有沒有道歉.1',
        '解釋.1', '引發同情(說自己有病).1', '藉口.1', '犯錯的類別.2', '表達目標人物/觀眾的感謝或諒解.2', '補償.2',
        '有沒有道歉.2', '解釋.2', '引發同情(說自己有病).2', '藉口.2', '犯錯的類別.3',
        '表達目標人物/觀眾的感謝或諒解.3', '補償.3', '有沒有道歉.3', '解釋.3', '引發同情(說自己有病).3', '藉口.3',
        '犯錯的類別.4', '表達目標人物/觀眾的感

In [314]:
sorry_df

,Youtuber,道歉日,犯錯的類別
0,阿神,2020/03/09,c
1,阿滴英文,2020/04/28,a
2,阿滴英文,2019/12/12,c
3,Joeman,2019/09/26,a
4,Joeman,2021/04/28,d
...,...,...,...
65,統神大戲院,2021/07/22,a
66,谷阿莫,2020/06/09,d
67,孫生又來了,2020/11/10,b
68,孫生又來了,2021/02/05,b


In [315]:
sorry_df = sorry_event[["youtuber", "date" , "犯錯的類別"]].copy()
sorry_df = sorry_df.rename(columns={'youtuber': 'Youtuber', 'date': '道歉日'})
sorry_df.loc[:, '道歉日'] = sorry_df['道歉日'].dt.strftime('%Y/%m/%d')

event_df = merged_with_bobo.copy()

ytr_df = ytr_type.rename(columns={"頻道(官方帳號)":"Youtuber"})


In [324]:
DAY_DELTA = 10
BEFORE_EVENT_COUNT = 1
final_df = pd.DataFrame()
event_ytrs = event_df['Youtuber'].unique() # merged_with_bobo 裡面所有的人
with open("../sorry_event_match.csv", "w+"):
    for ytr in sorry_df['Youtuber'].unique(): # google sheet 裡面所有的人

        if ytr not in event_ytrs:   # 如果有人不在 merged_with＿bobo，跳過
            continue

        sorry_date_list = sorry_df[sorry_df['Youtuber'] == ytr]['道歉日'].tolist() # list with sorry date

        # iterate each sorry date
        _focus_event_df = event_df[event_df['Youtuber'] == ytr]

        # 每個人自己的道歉日
        for sorry_day in sorry_date_list:
            
            _target_df = _focus_event_df[_focus_event_df['期間']
                                         < sorry_day][-BEFORE_EVENT_COUNT:]
            _target_df["道歉日"] = sorry_day

            # _target_df["犯錯的類別"] = 
            final_df = pd.concat([final_df, _target_df], axis=0)
            
final_df = sorry_df.merge(final_df, on=['Youtuber', '道歉日'])
final_df = final_df.merge(ytr_df, on=["Youtuber"])
final_df['sub'] = final_df['sub'] / 100000


In [325]:
final_df.columns

Index(['Youtuber', '道歉日', '犯錯的類別', '期間', '正面情緒', '負面情緒', '正面情緒(前一天)',
       '正面情緒(前二天)', '正面情緒(後一天)', '正面情緒(後二天)', '負面情緒(前一天)', '負面情緒(前二天)',
       '負面情緒(後一天)', '負面情緒(後二天)', '當天正負情緒比', '前一天正負情緒比', '前二天正負情緒比', '後一天正負情緒比',
       '後二天正負情緒比', '當天情緒比變化量', '隔天情緒比變化量', '前一後一情緒變化量', '前二後二情緒變化量', '聲量',
       '前一天聲量', '前二天聲量', '後一天聲量', '後二天聲量', '當天聲量變化量', '隔天聲量變化量', '前一後一聲量變化量',
       '前二後二聲量變化量', 'Dcard', 'Dcard內文', 'PTT', 'PTT內文', 'apple_news',
       'apple_news內文', 'ettoday', 'ettoday內文', 'Mobile01', 'Mobile01內文',
       'opview', 'g/b', 'sub', 'category', 'sorry', 'Unnamed: 1',
       '訂閱數(祐琳code)', 'Youtuber分類(祉均)', '代名詞', '道歉事件', 'Unnamed: 6',
       'Unnamed: 7', 'YTR 類別', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14'],
      dtype='object')

In [326]:
final_df[['Youtuber', '道歉日', 'YTR 類別', '犯錯的類別', 'sub', 'g/b', '前二後二情緒變化量', '期間']
         ].to_csv("../sorry_event_match.csv", index=False)


In [327]:
pd.read_csv("../sorry_event_match.csv")

,Youtuber,道歉日,YTR 類別,犯錯的類別,sub,g/b,前二後二情緒變化量,期間
0,阿滴英文,2020/04/28,知識教育類,a,26.90,good,0.149665,2020/04/12
1,Joeman,2021/04/28,生活類,d,22.20,good,-1.053086,2021/03/17
2,木曜4超玩,2021/07/04,生活類,b,21.90,good,2.181818,2021/07/01
3,木曜4超玩,2020/03/03,生活類,b,21.90,good,0.828571,2020/02/19
4,黃氏兄弟,2020/06/03,生活類,c,18.60,good,-9.678030,2020/05/30
5,WACKYBOYS 反骨男孩,2020/06/09,搞笑類,b,15.60,good,-0.030672,2020/06/05
6,滴妹,2021/11/14,生活類,d,14.20,good,2.000000,2021/10/27
7,放火 Louis,2020/10/24,搞笑類,c,12.40,bad,-0.038324,2020/10/23
8,愛莉莎莎 Alisasa,2020/04/09,生活類,a,11.80,bad,0.800419,2020/04/03
9,愛莉莎莎 Alisasa,2021/02/15,生活類,a,11.80,bad,-0.441204,2021/02/14
